In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
print("Started")
import numpy as np, pandas as pd
print("Pandas version - ", pd.__version__)
from pandas import MultiIndex, Int16Dtype

try:
    import seaborn as sns
except:
    !pip install seaborn
    import seaborn as sns
import matplotlib.pyplot as plt
import time
#import cupy, cudf
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV

# Importing RFE and LinearRegression
try:
    import statsmodels.api as sm
except:
    !python -m pip install statsmodels
    import statsmodels.api as sm

from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn import linear_model,tree,svm

try:
    import pickle
except:
    !pip install pickle5
    import pickle

from sklearn.metrics import mean_absolute_error
print("All necessary libraries imported")


In [3]:

traindf = pd.read_csv('../input/ventilator-pressure-prediction/train.csv')
testdf = pd.read_csv('../input/ventilator-pressure-prediction/test.csv')
sample_sub = pd.read_csv('../input/ventilator-pressure-prediction/sample_submission.csv')

print(traindf.shape, " - Training dataset shape")
print(testdf.shape, " - Testing dataset shape")
print(sample_sub.shape, " - Sample Submission dataset shape")

In [4]:
traindf.columns

# Adding feature

traindf['time_delta'] = traindf.groupby('breath_id')['time_step'].diff()
#testdf['time_delta'] = testdf.groupby('breath_id')['time_step'].diff()

In [5]:
# Adding features in the dataframe

def add_features(df):
    #https://www.kaggle.com/code/papernist/google-brain
    df = df.copy()
    
    df_group = df.groupby(['breath_id'])
    
    
    feature_list = ['u_in', 'time_step', 'cross']
    
    df['cross']= df['u_in'] * df['u_out']
    df['area_out']= df['time_step'] * df['u_out']
    df['area'] = df['time_step'] * df['u_in']
    
    # add
    df['air_flow_rate'] = df['u_out'] - (df['u_in']/100.0)
    df['air_flow_area'] = df['air_flow_rate'] * df['time_step']
    print("Step-1...Completed")
    
    # cumsum cummean
    df['one'] = 1
    df['count'] = df_group['one'].cumsum()
    for feature in feature_list:
        df[f'{feature}_cumsum'] = df_group[feature].cumsum()
        df[f'{feature}_cummean'] = df[f'{feature}_cumsum'] / df['count']
        
    print("Step-2 cumsum cummean ...Completed")
    # lagging
    use_lags = 4
    for lag in range(1, use_lags+1):
        for feature in feature_list:
            # lag 
            df[f'{feature}_lag_{lag}'] = df_group[feature].shift(lag)
            # inverse lag
            df[f'{feature}_lag_inverse_{lag}'] = df_group[feature].shift(-lag)

            # dif lag
            df[f'{feature}_lag_diff_{lag}'] = df[feature] - df[f'{feature}_lag_{lag}']

            # dif inverse lag
            df[f'{feature}_lag_inverse_diff_{lag}'] = df[feature] - df[f'{feature}_lag_inverse_{lag}']

            df = df.drop(columns=[f'{feature}_lag_{lag}', f'{feature}_lag_inverse_{lag}'])
        
    df = df.fillna(0)
    print("Step-3 lagging ...Completed")
    
    
    df['R'] = df['R'].astype(str)
    df['C'] = df['C'].astype(str)
    df['R__C'] = df["R"].astype(str) + '__' + df["C"].astype(str)
    df = pd.get_dummies(df)
    df = df.fillna(0)
    print("Step-4 categorical ...Completed")
    
    rolling_list = [15]
    for roll in rolling_list:
        for feature in feature_list:
            df[[f'{roll}_{feature}_sum',f'{roll}_{feature}_min',
                f'{roll}_{feature}_max',f'{roll}_{feature}_mean']] = (df_group[feature]\
                                                                  .rolling(window=roll,min_periods=1)\
                                                                  .agg({f'{roll}_{feature}_sum':'sum',
                                                                        f'{roll}_{feature}_min':'min',
                                                                        f'{roll}_{feature}_max':'max',
                                                                        f'{roll}_{feature}_mean':'mean'})\
                                                                   .reset_index(level=0,drop=True))
    
    print("Step-5 Sliding window...Completed")
    print()
    
    df = df.fillna(0)
    df = df.drop(['id', 'breath_id','one','count'], axis=1)
    
    return df.astype(np.float16)

In [6]:
%%time
train = add_features(traindf)
#test = add_features(testdf)

print("*"*70)
print(train.shape, " - Feature added training dataset shape")
#print(test.shape, " - Features added testing dataset shape")

In [7]:
binary_col = []
num_col = []

for columnname in train.columns:
  if len(set(train[columnname])) != 2:
    num_col.append(columnname)
  else:
    binary_col.append(columnname)

train_numcol = num_col.copy()
train_numcol.remove('pressure')

print(len(binary_col), "Number of Binary Categorical Columns")
print(len(num_col),"Number of Numerical Columns")
print(len(binary_col)+len(num_col), "Sum of Binary & Numerical Columns")

In [8]:
df_train, df_test = train_test_split(train, train_size = 0.7, test_size = 0.3, random_state = 100)
print("Split Done")

from sklearn import metrics
from sklearn import preprocessing
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV, cross_val_score, cross_val_predict, train_test_split

trainsplitcopy = df_train.copy()
testsplitcopy = df_test.copy()
traincopy = train.copy()

y_train = trainsplitcopy.pop('pressure')
X_train = trainsplitcopy

y_test = testsplitcopy.pop('pressure')
X_test = testsplitcopy

y_train_nosplit = traincopy.pop('pressure')
X_train_nosplit = traincopy



print("*"*70)
print("Splitting the data into X train & Y train")
print(X_train.shape, "X train shape")
print(y_train.shape, "Y train shape")

print("*"*70)
print("Splitting the data into X test & Y test")
print(X_test.shape, "X test shape")
print(y_test.shape, "Y test shape")

print("*"*70)
print("No Split Data")
print(X_train_nosplit.shape, "X train shape")
print(y_train_nosplit.shape, "Y train shape")


In [9]:
#train2 = train.copy()

In [10]:
#Scaling the features

scaler = MinMaxScaler().fit(X_train_nosplit[train_numcol])
train_scaled_nosplit = pd.DataFrame(scaler.transform(X_train_nosplit[train_numcol]),
                              columns = X_train_nosplit[train_numcol].columns)
train_scaled_nosplit.head()

In [11]:
#We will save the model performance metrics in a DataFrame
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import KFold, cross_val_score
import pickle
import numpy as np
Model = []
RMSE = []
R_sq = []
cv = KFold(5, random_state = 1,shuffle=True)

#Creating a Function to append the cross validation scores of the algorithms
def input_scores(name, model, x, y):
    Model.append(name)
    ts1 = time.time()
    print(name," - Started")
    rmse_tmp = np.sqrt((-1) * cross_val_score(model, x, y, cv=cv, scoring='neg_mean_squared_error').mean())
    RMSE.append(rmse_tmp)
    print(name, " - RSME - ", rmse_tmp)
    #Pickle Dump
    #file_name = f"{name}.sav"
    pickle.dump(model,open(f"{name}.h5","wb"))
    #pickle.dump(model,open(file_name,'wb'))
    #with open(f'{file_name[:-4]}','wb') as files:
        #pickle.dump(model,files)
    print(name," - Saved")
    
    R_sq_temp = cross_val_score(model, x, y, cv=cv, scoring='r2').mean()
    R_sq.append(R_sq_temp)
    print("R_sq - ", R_sq_temp)
    ts2 = time.time()
    print("Time taken (in mins) - ",(ts2-ts1)/60)
    print("*-*"*20)

In [ ]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import (RandomForestRegressor, GradientBoostingRegressor, 
                              AdaBoostRegressor)

names = ['Linear Regression', 'Ridge Regression', 'Lasso Regression',
         'K Neighbors Regressor', 'Decision Tree Regressor', 
         'Random Forest Regressor', 'Gradient Boosting Regressor',
         'Adaboost Regressor']
models = [LinearRegression(), Ridge(), Lasso(),
          KNeighborsRegressor(), DecisionTreeRegressor(),
          RandomForestRegressor(), GradientBoostingRegressor(), 
          AdaBoostRegressor()]

#Running all algorithms
for name, model in zip(names, models):
    input_scores(name, model,X_train_nosplit, y_train_nosplit)

In [ ]:
evaluation = pd.DataFrame({'Model': Model,
                           'RMSE': RMSE,
                           'R Squared': R_sq})
print("FOLLOWING ARE THE TRAINING SCORES: ")
evaluation